# 126 Project Traffic Simulator

Let the road be length $n$. Cars start at position 0 and reach their destination at position $n-1$. Each car has an exponential "clock" with parameter $\gamma$, and every time their clock goes off they move forward with probability $p$. If a car is directly in front of them, they do not move regardless. A car enters the road at position 0 at a rate of $Exp(\alpha)$ and exit the road at position $n-1$ with a rate of $Exp(\beta)$.

### Set global variables:

In [ ]:
n = 6  # length of path
alpha = .4  # exponential param for car entrance
beta = .8  # exponential param for car exit
gamma = .8  # exponential param for car movement
p = .7  # bernoulli param for car moving / stopped

seed = 454

### Define particle and environment classes:

In [ ]:
import time
import sys
import numpy as np

class Particle:
    def __init__(self, gamma=gamma, p=p):
        self.gamma = gamma
        self.p = p
        self._position = 0
        
    def update_position(self, value=1):
        self._position += value
        
    def get_position(self):
        return self._position
        
class Environment:
    def __init__(self, n=n, alpha=alpha, beta=beta):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.particles = []
        self.occupied_positions = [0 for _ in range(n)]
        
    def run_simulation(self, time_len=60):
        sys.stdout.write("\r{}".format(self.occupied_positions))
        sys.stdout.flush()
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/alpha)
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle()
        self.particles.append(particle)
        self.occupied_positions[0] = 1
        
        while curr_time < time_len:
            sys.stdout.write("\r{}".format(self.occupied_positions))
            sys.stdout.flush()
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.alpha + self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha, self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                if self.occupied_positions[0]:
                    continue
                else:
                    new_particle = Particle()
                    self.particles.append(new_particle)
                    self.occupied_positions[0] = 1
            elif particle == 'exit':
                if self.occupied_positions[n-1]:
                    self.particles.pop(0)
                    self.occupied_positions[n-1] = 0
                else:
                    continue
            else:
                particle_pos = particle.get_position()
                if particle_pos == n - 1 or self.occupied_positions[particle_pos + 1] == 1:
                    continue
                else:
                    self.occupied_positions[particle_pos] = 0
                    self.occupied_positions[particle_pos + 1] = 1
                    particle.update_position()
        print('\n done')

### Run simulator:

In [ ]:
env = Environment()
env.run_simulation()